# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
from unityagents import UnityEnvironment
from collections import deque

from scripts.model import * 
from scripts.ddpg_agent import *

import numpy as np
import torch

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

n this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

Run the code cell below to print some information about the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step. A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
env = UnityEnvironment(file_name="../Tennis_Linux/Tennis.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 2. MADDPG 



In [3]:
def ddpg(agent_1, n_episodes=2000, max_t=1000, print_every=100, consec_episodes=100, goal_score=0.5):
    
    scores_deque = deque(maxlen=consec_episodes)
    score = []
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name] 
        agent_1.reset()

        states = env_info.vector_observations 
        scores = np.zeros(num_agents)
                
        for timestep in range(max_t):

            actions = agent_1.act(states)
            
            env_info = env.step(actions)[brain_name]            # send all actions to the environment
            next_states = env_info.vector_observations          # get next state (for each agent)
            rewards = env_info.rewards                          # get reward (for each agent)
            
            dones = env_info.local_done                         # see if episode finished

            agent_1.step(states, actions, rewards, next_states, dones, timestep)
            
            
            scores += env_info.rewards                           # update the score (for each agent)
            states = next_states                                # roll over states to next time step
            
            if np.any(dones):                                   # exit loop if episode finished
                break
                
        scores_deque.append(scores)
        score.append(scores)
        
        #print('\rEpisode {}\tAverage Score: {:.5f}\t Max over agents {}'.format(i_episode, np.mean(scores_deque)),np.max(scores), end="")
        
        if i_episode % 10 == 0:
            print('Episode {}\tMax Reward: {:.3f}\tAverage Reward: {:.3f}\t'.format(
                i_episode, np.max(scores), np.mean(scores_deque)), np.max(scores_deque))

            
        if np.mean(scores_deque) >= goal_score:
                print('\n Requirements Satisfied in ' + str(i_episode) + ' episodes')
                torch.save(agent.actor_local.state_dict(), "agen_weights.pth")
                torch.save(agent.critic_local.state_dict(), "critic_weights.pth")
                
                break
            
    return scores, score

In [ ]:
env = UnityEnvironment(file_name="../Tennis_Linux/Tennis.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

env_info = env.reset(train_mode=True)[brain_name]
agent_1 = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, agents_number=2, random_seed=2)

num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size

scores, score = ddpg(agent_1)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


/home/ivan/Desktop/Udacity/MADDPG-Unity-Env/scripts/ddpg_agent.py:112: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(),1)


Episode 10	Max Reward: 0.000	Average Reward: 0.005	 0.10000000149011612
Episode 20	Max Reward: 0.000	Average Reward: 0.000	 0.10000000149011612
Episode 30	Max Reward: 0.000	Average Reward: -0.002	 0.10000000149011612
Episode 40	Max Reward: 0.000	Average Reward: -0.002	 0.10000000149011612
Episode 50	Max Reward: 0.000	Average Reward: -0.003	 0.10000000149011612
Episode 60	Max Reward: 0.000	Average Reward: -0.003	 0.10000000149011612
Episode 70	Max Reward: 0.000	Average Reward: -0.001	 0.10000000149011612
Episode 80	Max Reward: 0.000	Average Reward: -0.001	 0.10000000149011612
Episode 90	Max Reward: 0.000	Average Reward: -0.002	 0.10000000149011612
Episode 100	Max Reward: 0.090	Average Reward: 0.001	 0.10000000149011612
Episode 110	Max Reward: 0.100	Average Reward: 0.004	 0.10000000149011612
Episode 120	Max Reward: 0.090	Average Reward: 0.008	 0.10000000149011612
Episode 130	Max Reward: 0.090	Average Reward: 0.012	 0.10000000149011612
Episode 140	Max Reward: 0.100	Average Reward: 0.015	 

In [ ]:
plt.plot(np.arange(0, len(score)), score)
plt.ylabel('Score achieved')
plt.xlabel('Episodes')
plt.show()